Train : 1024x1024 , Test : 224x224
최대공약수 32

import

In [34]:
import pandas as pd
import numpy as np
import cv2
import os

Train image

In [3]:
df=pd.read_csv('./train.csv')
img_path = df.iloc[:, 1]

Split train_img to 4X4 (size : 224x224)

In [ ]:
train_directory=os.listdir('./train_img')

for filename in train_directory:
    empty_matrix=[[[[0 for _ in range(224)] for _ in range(224)] for _ in range(3)] for _ in range(16)]
#     print(filename)
    image=cv2.imread('./train_img/'+filename)
    count=0
    for i in range(4):
        for j in range(4):
            empty_matrix[count]=image[224*i:224*i+223,224*j:224*j+223,0:3]
            cv2.imwrite('./preprocessed_train_img/{path}{by}{count}.jpg'.format(path=filename,by='_',count=count),empty_matrix[count])
            count=count+1